In [1]:
import numpy as np
import matplotlib.pyplot as mp
from random import random

EL OBJETIVO ES OBTENER LAS CURVAS DE HISTÉRESIS **SIN CONSIDERAR** EL CAMPO GNERADO POR EL MOMENTO DIPOLAR MAGNÉTICO DE LAS NANOPARTÍCULAS.


In [6]:
#definimos una función para realizar el producto punto dentro de los vectores que conforman las matrices

# AQUÍ PODEMOS CAMBIAR EL DÍAMETRO DE LAS NANOPARTICULAS
# Y LA FRECUENCIA DE OSCILACIÓN DEL CAMPO MAGNÉTICO

cantidad_de_np = 10

diametro = 12*10**-9    #DIAMETRO DE LA NANOPARTÍCULA en metros
f = 5*10**-7           #Radio del lisosoma
B_max = 2.5            #Tesla

#Funciones definidas:
#  p_punto(m,e,i,j)
#  campo_externo(t)
#  campo_total(t)
#  energia(theta,i)
#  derivada_ener(x,i)
#  raices(i)
#  ener_raices(j)
#  probability(i)



def p_punto(m,e,i,j):    #definimos el producto punto para mtiplicar filas de matrices
    
    s = 0

    for k in range(0,3):
        s = s + m[i,k] * e[j,k]
        
    return s




u_0   = 12.57 * 10**(-7) # H / m

def campo_externo(t):       #Definimos la función del campo externo
    #w = 628318.5307179586
    w = 0.5     #valor de prueba
    H = B_max  * np.sin(w*t)
    campo = [H,0,0]
    return campo



## POSICION DE LAS NANOPARTICULAS



position =  np.matrix(np.empty(shape=(0,3), dtype=float))

#Estamos considerando las posiciones de una esfera ubicada en (1,1,1)
#Y con el radio del lisosoma
n = 0
while n < cantidad_de_np:
    a = 2*random()
    b = 2*random()
    c = 2*random()
    if (a-1)**2 + (b-1)**2 + (c-1)**2 <= 1:
    
        position = np.insert(position, n, [a*f , b*f, c*f], axis = 0)
        n = n + 1


## DIRECCIÓN DE LA MAGNETIZACIÓN INICIAL y DE EASY AXIS
## AMBAS DIRECCIONES SE OBTIENEN COMO VECTORES UNITARIOS


easy_axis = np.matrix(np.empty(shape=(0,3), dtype=float))

for i in range(0,cantidad_de_np):
    a = random()
    b = random()
    c = random()
    d = (a**2 + b**2 + c**2)**0.5
    easy_axis = np.insert(easy_axis, i, [a/d , b/d, c/d], axis = 0)

    
    
magnetizacion = np.matrix(np.empty(shape=(0,3), dtype=float))
    
for i in range(0,cantidad_de_np):
    a = random()
    b = random()
    c = random()
    d = (a**2 + b**2 + c**2)**0.5
    magnetizacion = np.insert(magnetizacion, i, [a/d , b/d, c/d], axis = 0)
    
#magnetizacion,position




def energia(theta,i):
    K_eff = 10**6           # J m^-3
    V     = (4/3)*np.pi*(diametro/2)**3      # m^3
    u_0   = 12.57 * 10**(-7) # H / m
    M_s   = 10**6            # A m^-1

    
    #campo = (campo_ext[i,]**2).sum() **0.5
    
    #return np.sin(theta)**2 - -2*np.cos(theta-np.pi/6)   E / K_b T
    #return campo
    return (K_eff*V)*np.sin(theta)**2 - (M_s*V*campo_ext[0])*np.cos(theta-angulo_mag[i])
    

def derivada_ener(x,i):
    h = 0.0000001
    d = (energia(x+h,i) - energia(x,i) )/ h
    return d


def raices(i):  #i es el número de nanopartícula
    raices = []
    
    
    tolerancia = 0.0001
    cota_de_error = np.pi/100
    
    h = 0.00001
    paso = 5/200
    
    K_B   = 1.38064852 * 10**(-23) # m^2 kg s^-2 K^-1
    T     = 300
    
    v_10, v_20 = 10**10,10**10
    
    t_step = 0.00005
    
    for j in range(0,200):    #comienza en 4 para evitar encontrar la raiz trivial (el máximo global)
        
        a = 4 - paso*j
        c = 4 - (1 + j)*paso
        d_1 = derivada_ener(a,i) 
        d_2 = derivada_ener(c,i)
        if (d_1 < 0 and d_2 > 0):
        #if (d_1 > 0 and d_2 < 0):
        
            x_0 = c
            
            
            
            for l in range (0,10):
                dderiv_ener = (derivada_ener(x_0 + h,i)-derivada_ener(x_0,i) )/h
                x_i = x_0 - derivada_ener(x_0,i)/dderiv_ener
                
                x_0 = x_i
                
            raices.append(x_0)
            
            
        elif (d_1 > 0 and d_2 < 0):
              
            x_0 = c
                        
            for l in range (0,10):
                dderiv_ener = (derivada_ener(x_0 + h,i)-derivada_ener(x_0,i) )/h
                x_i = x_0 - derivada_ener(x_0,i)/dderiv_ener
                
                x_0 = x_i
            raices.append(x_0)
                
    return raices                
            
    
def ener_raices(j):
    energies = []
    
    for i in range(0,2):
        energies.append(energia(raices[i],j))
        
    return energies
            
        
def probability(i):
    
    
    if np.shape(raices) == (3,):
        #CALCULAMOS v_1 Y v_2
        v_2 = v_10 * np.exp(-(energies[1] - energies[0])/(K_B*T))
        
        v_1 = v_20 * np.exp(-(energies[1] - energies[2])/(K_B*T))
        
        P_21 = v_2/(v_1 + v_2) *(1 - np.exp(-(v_1 + v_2)*t_step))
        #P_21 =  (np.exp(-(v_1 + v_2)*t_step))
        
        return P_21
    


In [8]:
campo_ext = campo_externo(6)

In [9]:
raices(4)

NameError: name 'angulo_mag' is not defined

In [ ]:
#Porque w = 0.5, consideramos que cada paso del algoritmo será 2*pi/(0.5*2000)
step = 2*np.pi/(0.5 * 2000)
t = 0

for i in range(0,2000):
    
    
    campo_dipol = campo_total(t)
    
    angulo_mag      = []       # Easy axis - magnetizacion   -> Theta
    angulo_posicion = []       # Easy axis - magnetic field  -> Phi
    
    for i in range(0,cantidad_de_np):
        w = campo_dipol[i,]
        q = np.matrix(w/(w**2).sum() **0.5)
        plano = np.cross(q,easy_axis[i,])
        
        
        ang_2 = np.arccos(p_punto(q,easy_axis[i,],0,0))     #Phi
        angulo_posicion.append(ang_2)
        
        ang_1 = np.arccos(p_punto(easy_axis,magnetizacion,0,i))
        angulo_mag.append(ang_1)
    
    
    
    for j in range(0,cantidad_de_np):
        
        maximos = raices(j)
        
        energies = ener_raices(j)
        
        P = probabilidad(j)
        
        if np.shape(raices) == (3,):
            
            if angulo_mag > maximos[1] and angulo_mag < maximos:
            
            angulo_mag[j] = raices[]
    
    
    
    t = t + step